In [1]:
import logging
import os
log_file = os.path.normpath("C:/Temp/bcu/log/import-eprints-dump.log")
file_mode = 'w'
logging.basicConfig(filename=log_file, filemode=file_mode, level=logging.INFO, format='%(asctime)s %(levelname)s : %(message)s')

In [2]:
# Load XML file
dump_file = os.path.normpath('C:/Temp/bcu/export_bcu_records.xml')

In [3]:
PRETTY_PRINT = False
NSMAP = {'ep': 'http://eprints.org/ep2/data/2.0'}

In [4]:
from lxml import etree as et
import json, psycopg2
conn = psycopg2.connect(host="localhost", port=5432, database="BCU", user="postgres", password="admin")
insert_sql = 'insert into items(id, type, xml_in) values(%s, %s, %s) on conflict(id) do update set type = excluded.type, xml_in = excluded.xml_in, xml_out = null'
conn.autocommit = True
cur = conn.cursor()
# Remove white space from XML nodes to make parsing work better in transformation
parser = et.XMLParser(remove_blank_text=True)
root = et.parse(dump_file, parser)
items = root.xpath('//ep:eprint', namespaces=NSMAP)
logging.info('Processing {} publications'.format(len(items)))
with conn:
    for item in items:
        xml_in = et.tostring(item, pretty_print=PRETTY_PRINT, encoding="UTF-8").decode("UTF-8")
        item_id = item.find('ep:eprintid', namespaces=NSMAP).text
        content_type = item.find('ep:type', namespaces=NSMAP).text
        logging.info('Processing publication {}'.format(item_id))
        cur.execute(insert_sql, (item_id, content_type, xml_in))
conn.close()
logging.info('Stored all publications')